In [ ]:
from stmpy import Driver, Machine
import ipywidgets as widgets
from IPython.display import display
import paho.mqtt.client as mqtt
import json

tasks = [1, 2, 3, 4, 5]
group = 2
task_answers ={
    "group": group,
    1 : "",
    2 : "",
    3 : "",
    4 : "",
    5 : ""
}


#Est. mqtt connection, student as publisher
mqttBroker = "mqtt.eclipseprojects.io"
client = mqtt.Client("Student_group-2")
client.connect(mqttBroker)

class Student:
    def __init__(self):
        print("Init - Student")
        self.counter = 0

        self.button_next_task = widgets.Button(description="Next task")
        self.button_next_task.on_click(self.next_task_pressed)

        self.button_help = widgets.Button(description="Help")
        self.button_help.on_click(self.help_pressed)

        self.text_area = widgets.Textarea(
            value='',
            placeholder='Your answer',
            description='Task {}: '.format(tasks[self.counter]),
            disabled=False
        )

        display(self.button_next_task)
        display(self.button_help)
        display(self.text_area)

    def show_tasks(self):
        self.stm.send('start_task')
        #publish start first task
        client.publish("Task", "{} {}".format(group, tasks[self.counter]))

    def task_func(self):
        if self.button_next_task.description == "Next task":
            if self.counter == len(tasks) - 1:
                self.button_next_task.description = "Submit tasks"
            else:
                self.button_next_task.description = "Next task"
            print("Show tasks nr: ", tasks[self.counter])
        else:
            print("Delivered")
            self.stm.send('deliver')
            # Publish delivered
            client.publish("Deliver", "{} delivered".format(group))
            encode_task_answers = json.dumps(task_answers, indent=2).encode('utf-8')
            client.publish("Deliver_save", encode_task_answers)

    def next_task_pressed(self, b):
        if self.text_area.value.strip() == "":
            print("You must submit answer to continue")
        else:
            task_answers[self.counter + 1] = self.text_area.value
            self.counter += 1
            self.stm.send('next_task')
            self.text_area.value = ''
            # Publish progress
            if self.counter < len(tasks):
                client.publish("Task", "{} {}".format(group, tasks[self.counter]))
                self.text_area.description = 'Task {}: '.format(tasks[self.counter])

    def help_pressed(self, b):
        if self.button_help.description == "Help":
            self.button_help.description = "Help finished"
            self.stm.send('help')
        else:
            self.button_help.description = "Help"
            self.stm.send('help_finished')
    
    def add_group_to_queue(self):
        print("Added to queue")
        # Publish that group needs help
        client.publish("Help", "{}".format(group))
        self.button_help.description = "Help finished"

    def remove_group_from_queue(self):
        print("Removed from queue")
        # Publish that group has received help 
        client.publish("Help_finished", "{}".format(group))
        self.button_help.description = "Help"

    def finished(self):
        self.stm.driver.stop()



#for student
s0 = {"source": "initial", "target": "display_tasks"}

s1 = {
    "trigger": "start_task",
    "source": "display_tasks",
    "target": "task"
}
s2 = {
    "trigger": "next_task",
    "source": "task",
    "target": "task"
}
s3 = {
    "trigger": "help",
    "source": "task",
    "target": "TA_assistance"
}
s4 = {
    "trigger": "t",
    "source": "task",
    "target": "TA_assistance"
}
s5 = {
    "trigger": "deliver",
    "source": "task",
    "target": "end"  #what is end?
}
s6 = {
    "trigger": "back_to_waiting",
    "source": "task",
    "target": "display_tasks"
}
s7 = {
    "trigger": "help_finished",
    "source": "TA_assistance",
    "target": "task"
}


#student states
display_tasks = {'name': 'display_tasks',
      'entry': 'show_tasks'
      }
task = {'name': 'task',
      'entry': 'start_timer("t", 100000000); task_func'
      }
TA_assistance = {'name': 'TA_assistance',
      'entry': 'stop_timer("t"); add_group_to_queue',
      'exit': 'remove_group_from_queue'
      }
end = {'name': 'end',
      'entry': 'finished'
      }


student = Student()
student_machine = Machine(name='stm_student', transitions=[s0, s1, s2, s3, s4, s5, s6, s7], obj=student, states=[display_tasks, task, TA_assistance, end])
student.stm = student_machine

driver = Driver()
driver.add_machine(student_machine)
driver.start()

#driver.stop()